In [ ]:
!pip install pytorch_lightning
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install wandb -qqq

In [ ]:
import wandb
wandb.login()

In [1]:
import json
import torch
from tqdm import tqdm
import torch.nn as nn
import os, glob, re
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import (T5ForConditionalGeneration,
                          AdamW,
                          T5TokenizerFast as token)

from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
pl.seed_everything(13)
print(torch.__version__)
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

Global seed set to 13


1.8.1+cu101


In [2]:
df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_qa.csv')
df[df.answer.isna()]

,question,text,answer,answer_start,answer_end,origin_text,origin_answer_start,origin_answer_end,len text,id


In [10]:
# wandb.init(project="ci", config={
#     "learning_rate": 0.0001,
#     "architecture": "T5",
#     'model': 't5-base',
#     "dataset": "Coleridge Initiative ",
#     'tex_max_len': 396,
#     'asw_max_len': 44,
#     'batch_size' : BATCH_SIZE,
#     'epoch':N_EPOCHS
# })
# config = wandb.config

BATCH = 6
EPOCHS =1

config={
    "learning_rate": 0.0001,
    "architecture": "T5",
    'model': 't5-base',
    "dataset": "Coleridge Initiative ",
    'tex_max_len': 396,
    'asw_max_len': 44,
    'batch_size' : BATCH,
    'epoch':EPOCHS,
    'device': 'cuda'
}

In [4]:
class CI_Dataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: token,
        config
    ):

        self.tokenizer = tokenizer
        self.data = data
        self.tex_max_len = config['tex_max_len']
        self.asw_max_len = config['asw_max_len']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index:int):
        txt = self.data.iloc[index]

        encode_txt = token(
            txt['question'],
            txt['text'],
            max_length = self.tex_max_len, 
            padding= 'max_length',
            truncation = 'only_second',
            return_attention_mask = True,
            add_special_tokens =True,
            return_tensors = 'pt'
            )
        
        encode_asw = token( 
            txt['answer'],
            max_length = self.asw_max_len,
            padding= 'max_length',
            truncation = True,
            return_attention_mask = True,
            add_special_tokens =True,
            return_tensors = 'pt'
            )
        labels = encode_asw['input_ids']
        labels[labels == 0] = -100

        return dict(
            question=txt['question'],
            text=txt['text'],
            answer=txt['answer'],
            input_ids=encode_txt['input_ids'].flatten(),
            attention_mask=encode_txt['attention_mask'].flatten(),
            labels = labels.flatten()
            )
        
class CI(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = T5ForConditionalGeneration.from_pretrained(config['model'], return_dict = True)

    def forward(self, input_ids, attention_mask, labels):
        out = self.model(input_ids = input_ids,
                    attention_mask = attention_mask,
                    labels = labels
                    )
        return out.loss, out.logits

In [5]:
MODEL = config['model']
token = token.from_pretrained(MODEL)
exampe_dataset = CI_Dataset(df, token, config)

for data in exampe_dataset:
    print(data['question'])
    print(data['text'])
    print(data['answer'])

    print(data['input_ids'][:10])
    print(data['attention_mask'][:10])    
    break

the impact of evaluation use on accounting programs performance an exploratory study
the quality of educational programs has been an object of debate and research around the world initiatives such as the program for international student assessment pisa and the trends in international mathematics and science study timss show that international organizations such as the organization for economic co operation and development oecd and the international association for the evaluation of educational achievement iea are trying to verify whether schools are adequately preparing their students by comparing their performances aiming to highlight the strengths and weaknesses among the educational systems of different countries higher education has also been the object of quality evaluations around the world ursin huusko aittola kiviniemi muhonen van kemenade pupius hardjono governmental and non governmental organizations have developed ways to certify institutional quality through evaluation or 

In [8]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# def loss_func(pred, target):
#     tmp = []
#     token = token.from_pretrained(MODEL)
#     for i in range(len(t)):
        
#         tt = np.where(t[i] != -100, t[i], 0)
#         decode = token.decode(tt, skip_special_tokens=True, clean_up_tokenization_spaces=True)          
#         tmp.append(jaccard(''.join(decode), target))
#     return tmp

def loss_func(pred, target, config):
    tmp = []
    MODEL = config['model']
    tokenizer = token.from_pretrained(MODEL)
    for i in range(len(pred)):
        out_decode = tokenizer.decode(np.argmax(pred[i], axis = 1),
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=True)       
        tmp.append(jaccard(''.join(out_decode), target[i]))
    return tmp


def train(model, data_loader, optimizer, config, scheduler = None):
    model.train()
    for _, txt in enumerate(data_loader):        
        input_ids=txt['input_ids'].to(config['device'])
        mask=txt['attention_mask'].to(config['device'])
        labels = txt['labels'].to(config['device'])
        optimizer.zero_grad()
        loss, out = model(input_ids, mask, labels)
        loss.backward()
        if scheduler is not None:
            scheduler.step()

def valid(model, data_loader,config):
    model.eval()
    losses = []
    asw = []
    loss_2 = []
    outs = []
    for _, txt in enumerate(data_loader):
        input_ids=txt['input_ids'].to(config['device'])
        mask=txt['attention_mask'].to(config['device'])
        labels = txt['labels'].to(config['device']) 
        answer=txt['answer']

        loss, out = model(input_ids, mask, labels)
        out = out.cpu().detach().numpy()
        loss2 = loss_func(out, answer, config)
        losses.append(loss.cpu().detach().numpy())
        loss_2.append(loss2)
        asw.append(answer)
        outs.append(out)

    return np.vstack(losses), np.vstack(asw), np.vstack(loss_2), np.vstack(outs)


def run(config):
    MODEL = config['model']

    df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_qa.csv')
    df_small = df.drop_duplicates(subset=['question']).reset_index(drop=True)
    train_df, val_df = train_test_split(df_small.head(100), random_state = 13, test_size = 0.1)
    tr = train_df.reset_index(drop=True)
    vl = val_df.reset_index(drop=True)

    tokenizer = token.from_pretrained(MODEL)

    tr_dataset = CI_Dataset(tr,tokenizer,config)
    vl_dataset = CI_Dataset(vl,tokenizer,config)

    tr_loader = DataLoader(tr_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4)
    vl_loader = DataLoader(vl_dataset, batch_size = 1, num_workers = 4)

    model = CI(config).to(config['device'])
    optimizer = AdamW(model.parameters(), lr = config['learning_rate'])
    bar =  tqdm(range(config['epoch']))
    for e in bar:
          
        train(model, tr_loader, optimizer, config)
        l, t, j, o = valid(model, vl_loader, config)
        print('Loss:', np.mean(l))
        bar.set_description(f'Jaccard Loss: {np.mean(j)}, Epoch: {e +1}')

    torch.cuda.empty_cache()
    return l,t, j, o

In [11]:
l, t, j, o = run(config)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Jaccard Loss: 0.35218253968253976, Epoch: 1:  50%|█████     | 1/2 [01:05<01:05, 65.48s/it]

Loss: 8.616283


Jaccard Loss: 0.35218253968253976, Epoch: 2: 100%|██████████| 2/2 [02:10<00:00, 65.36s/it]

Loss: 8.616283


In [ ]:
l

In [31]:
jaccard('adni', 'andi')

0.0

In [61]:
np.argmax(o[1], axis = 1)

array([32099,    13,  2913,   585,    11,   606,  6397,  6397,     3,
           3,     3, 32099, 32099, 32099, 32099, 32099, 32099, 32099,
       32099, 32099, 32099, 32099, 32099, 32099, 32099, 32099, 32099,
       32099, 32099, 32099, 32099, 32099, 32099, 32099, 32099, 32099,
       32099, 32099, 32099, 32099, 32099, 32099, 32099, 32099])

In [27]:
token.decode(np.argmax(o[-2], axis = 1), skip_special_tokens=True, clean_up_tokenization_spaces=True)

'jd andi als      '

In [18]:
t

array([['national assessment of education progress'],
       ['survey of industrial research and development'],
       ['early childhood longitudinal study'],
       ['coastal change analysis program'],
       ['trends in international mathematics and science study'],
       ['nass census of agriculture'],
       ['north american breeding bird survey'],
       ['world ocean database'],
       ['adni'],
       ['coastal change analysis program']], dtype='<U53')